## Etapa 1: Processamento e Consolidação dos Dados

Neste notebook, vamos ler os três ficheiros JSON brutos (`vagas`, `prospects`, `applicants`), limpá-los e padronizá-los na origem, uni-los numa única base de dados e aplicar os filtros de negócio.

In [1]:
import pandas as pd
import json
import re

print("--- ETAPA 1: PROCESSAMENTO E CONSOLIDAÇÃO DOS DADOS ---")

--- ETAPA 1: PROCESSAMENTO E CONSOLIDAÇÃO DOS DADOS ---


### Bloco 1: Processar `vagas.json`

In [2]:
print("\nProcessando vagas.json...")
with open('../data/raw/vagas.json', 'r', encoding='utf-8') as f:
    vagas_data = json.load(f)

processed_vagas = []
for vaga_id, data in vagas_data.items():
    record = {'id_vaga': vaga_id}
    if isinstance(data.get('informacoes_basicas'), dict):
        for key, value in data['informacoes_basicas'].items():
            record[f'info.{key}'] = value
    if isinstance(data.get('perfil_vaga'), dict):
        for key, value in data['perfil_vaga'].items():
            record[f'perfil.{key}'] = value
    processed_vagas.append(record)

vagas_df = pd.DataFrame(processed_vagas)

# --- LIMPEZA NA ORIGEM ---
# Trata valores nulos (null) e espaços em branco nos campos de idioma
if 'perfil.nivel_ingles' in vagas_df.columns:
    vagas_df['perfil.nivel_ingles'] = vagas_df['perfil.nivel_ingles'].fillna('Nenhum').str.strip()
if 'perfil.nivel_espanhol' in vagas_df.columns:
    vagas_df['perfil.nivel_espanhol'] = vagas_df['perfil.nivel_espanhol'].fillna('Nenhum').str.strip()
print("Níveis de idioma das vagas foram limpos e padronizados.")
# --- FIM DA LIMPEZA ---

print(f"vagas.json processado. {len(vagas_df)} vagas encontradas.")


Processando vagas.json...
Níveis de idioma das vagas foram limpos e padronizados.
vagas.json processado. 14081 vagas encontradas.


### Bloco 2: Processar `prospects.json`

In [3]:
print("\nProcessando prospects.json...")
with open('../data/raw/prospects.json', 'r', encoding='utf-8') as f:
    prospects_data = json.load(f)

prospects_list = []
for vaga_id, data in prospects_data.items():
    for prospect in data.get('prospects', []):
        if prospect: 
            prospect_info = {
                'id_vaga': vaga_id,
                **prospect
            }
            prospects_list.append(prospect_info)

prospects_df = pd.DataFrame(prospects_list).add_prefix('prospect.')
prospects_df.rename(columns={'prospect.id_vaga': 'id_vaga'}, inplace=True)
print(f"prospects.json processado. {len(prospects_df)} candidaturas encontradas.")


Processando prospects.json...
prospects.json processado. 53759 candidaturas encontradas.


### Bloco 3: Processar `applicants.json` e Aplicar Filtros

In [4]:
print("\nProcessando applicants.json...")
applicants_raw_df = pd.read_json('../data/raw/applicants.json', orient='index')
applicants_raw_df.index.name = 'id_candidato'

# Achatamento (flattening) dos dados do candidato
nested_columns = [
    'infos_basicas', 'informacoes_pessoais', 'informacoes_profissionais', 
    'formacao_e_idiomas', 'cargo_atual'
]
df_parts = [applicants_raw_df.drop(columns=nested_columns, errors='ignore')]

for col in nested_columns:
    if col in applicants_raw_df.columns:
        series_no_na = applicants_raw_df[col].dropna()
        if not series_no_na.empty and isinstance(series_no_na.iloc[0], dict):
            normalized_part = pd.json_normalize(applicants_raw_df[col]).add_prefix(f"{col}.")
            normalized_part.index = applicants_raw_df.index
            df_parts.append(normalized_part)

applicants_df = pd.concat(df_parts, axis=1)
applicants_df.reset_index(inplace=True)
print("Dados dos candidatos foram achatados (flattened).")

# --- LIMPEZA NA ORIGEM ---
# Trata valores nulos (null) e espaços em branco nos campos de idioma do candidato
if 'formacao_e_idiomas.nivel_ingles' in applicants_df.columns:
    applicants_df['formacao_e_idiomas.nivel_ingles'] = applicants_df['formacao_e_idiomas.nivel_ingles'].fillna('Nenhum').str.strip()
if 'formacao_e_idiomas.nivel_espanhol' in applicants_df.columns:
    applicants_df['formacao_e_idiomas.nivel_espanhol'] = applicants_df['formacao_e_idiomas.nivel_espanhol'].fillna('Nenhum').str.strip()
print("Níveis de idioma dos candidatos foram limpos e padronizados.")
# --- FIM DA LIMPEZA ---

print(f"{len(applicants_df)} candidatos brutos carregados.")

# Filtro de negócio: manter apenas candidatos que mencionam 'qualificações' no CV
print("Aplicando filtro de qualificações no CV...")
initial_candidates = len(applicants_df)
if 'cv_pt' in applicants_df.columns:
    mask = applicants_df['cv_pt'].notna() & applicants_df['cv_pt'].str.contains(
        'qualificaç(?:ão|ões)',
        case=False, 
        na=False, 
        regex=True
    )
    applicants_filtered_df = applicants_df[mask].copy()
    print(f"{initial_candidates - len(applicants_filtered_df)} candidatos removidos.")
    print(f"{len(applicants_filtered_df)} candidatos válidos restantes.")
else:
    print("Aviso: Coluna 'cv_pt' não encontrada. O filtro de qualificações não foi aplicado.")
    applicants_filtered_df = applicants_df


Processando applicants.json...
Dados dos candidatos foram achatados (flattened).
Níveis de idioma dos candidatos foram limpos e padronizados.
42482 candidatos brutos carregados.
Aplicando filtro de qualificações no CV...
34447 candidatos removidos.
8035 candidatos válidos restantes.


### Bloco 4: Unificar os Dados

In [5]:
print("\nUnificando os dados...")
# Garante que as chaves de união sejam do mesmo tipo (string)
prospects_df['id_vaga'] = prospects_df['id_vaga'].astype(str)
vagas_df['id_vaga'] = vagas_df['id_vaga'].astype(str)
prospects_df['prospect.codigo'] = prospects_df['prospect.codigo'].astype(str)
applicants_filtered_df['id_candidato'] = applicants_filtered_df['id_candidato'].astype(str)

dados_consolidados = pd.merge(prospects_df, vagas_df, on='id_vaga', how='left')
dados_consolidados = pd.merge(dados_consolidados, applicants_filtered_df, left_on='prospect.codigo', right_on='id_candidato', how='inner')
print("Unificação concluída.")


Unificando os dados...
Unificação concluída.


### Bloco 5: Criar a Variável-Alvo (`match`)

In [6]:
print("\nCriando a variável-alvo 'match'...")
dados_consolidados['match'] = (dados_consolidados['prospect.situacao_candidado'] == 'Contratado pela Decision').astype(int)
print("Variável-alvo criada.")


Criando a variável-alvo 'match'...
Variável-alvo criada.


### Bloco Final: Análise e Salvamento

In [7]:
print("\n--- BASE DE DADOS CONSOLIDADA ---")
print(f"Dimensões da base final: {dados_consolidados.shape[0]} linhas, {dados_consolidados.shape[1]} colunas")

print("\nDistribuição da variável-alvo:")
print(dados_consolidados['match'].value_counts())

dados_consolidados.to_json('../data/processed/dados_consolidados.json', orient='records', lines=True, force_ascii=False)
print("\nArquivo '../data/processed/dados_consolidados.json' salvo com sucesso!")
print("\n--- ETAPA 1 CONCLUÍDA ---")


--- BASE DE DADOS CONSOLIDADA ---
Dimensões da base final: 13846 linhas, 108 colunas

Distribuição da variável-alvo:
match
0    13073
1      773
Name: count, dtype: int64

Arquivo '../data/processed/dados_consolidados.json' salvo com sucesso!

--- ETAPA 1 CONCLUÍDA ---
